In [1]:
import pandas as pd
import datetime

In [3]:
orga_df = pd.read_csv('../data/cleaned/organisation.csv',
                      sep='|',
                      encoding='Latin-1')

FileNotFoundError: File b'../data/cleaned/organisation.csv' does not exist

In [3]:
equipement_df = pd.read_csv('data/cleaned/equipment.csv',
                           sep='|',
                           encoding='Latin-1', parse_dates=['INSTALL_DATE', 'RACHAT_DATE'])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
contrat_history_df = pd.read_csv('data/processed/contract_history.csv',
                           sep='|',
                           encoding='Latin-1', parse_dates=['CRE_DATE', 'UPD_DATE', 'DATE_RESILIATION', 'DATE_DEBUT', 'DATE_FIN'])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
for c in ['CRE_DATE', 'UPD_DATE']:
    contrat_history_df['{}_date'.format(c)] = pd.to_datetime(contrat_history_df[c].map(lambda x: datetime.datetime.date(x)))

In [6]:
intervention_test_df = pd.read_csv('data/final/intervention_test.csv',                              
                           sep='|',
                           encoding='Latin-1',
                           parse_dates=['SCHEDULED_START_DATE', 'SCHEDULED_END_DATE', 'CRE_DATE_GZL'])
print(intervention_test_df.shape, intervention_test_df.INCIDENT_NUMBER.nunique(), intervention_test_df.INSTANCE_ID.nunique())

(226770, 19) 226770 226770


In [7]:
intervention_train_df = pd.read_csv('data/final/intervention_train.csv',                              
                           sep='|',
                           encoding='Latin-1',
                           parse_dates=['SCHEDULED_START_DATE', 'SCHEDULED_END_DATE', 'CRE_DATE_GZL'])
print(intervention_train_df.shape, intervention_train_df.INCIDENT_NUMBER.nunique(), intervention_train_df.INSTANCE_ID.nunique())

(1048030, 20) 1048030 462663


In [8]:
intervention_history_df = pd.read_csv('data/processed/intervention_history.csv',                              
                           sep='|',
                           encoding='Latin-1',
                           parse_dates=['DATE_SAISIE_RETOUR', 'SCHEDULED_START_DATE', 'SCHEDULED_END_DATE', 'ACTUAL_START_DATE', 'ACTUAL_END_DATE', 'CRE_DATE_GZL'])
print(intervention_history_df.shape, intervention_history_df.INCIDENT_NUMBER.nunique(), intervention_history_df.INSTANCE_ID.nunique())

(5171308, 30) 5171308 795394


In [9]:
nature_code_eau_chaude = pd.read_csv('data/final/nature_code_eau_chaude.csv', sep='|', encoding='Latin-1')

In [10]:
nature_code_energie = pd.read_csv('data/final/nature_code_energie.csv', sep='|', encoding='Latin-1')

In [11]:
nature_code_fonction = pd.read_csv('data/final/nature_code_fonction.csv', sep='|', encoding='Latin-1')

In [12]:
nature_code_installation = pd.read_csv('data/final/nature_code_installation.csv', sep='|', encoding='Latin-1')

In [13]:
nature_code_specification = pd.read_csv('data/final/nature_code_specification.csv', sep='|', encoding='Latin-1')

In [14]:
def prepare_data(data_):
    data = data_.merge(equipement_df, how='left', on='INSTANCE_ID').merge(orga_df, how='left', left_on='ORGANISATION_ID', right_on='L2_ORGANISATION_ID')
    contrat_history_s = data[['INCIDENT_NUMBER', 'INSTANCE_ID','CRE_DATE_GZL']].merge(contrat_history_df).query('CRE_DATE_GZL>=UPD_DATE_date')
    contrat_history_s = contrat_history_s.sort_values(['INCIDENT_NUMBER', 'UPD_DATE'], ascending=[True,False]).drop_duplicates(keep='first', subset=['INCIDENT_NUMBER'])
    data = data.merge(contrat_history_s, how='left').merge(nature_code_eau_chaude, how='left')
    data = data.merge(nature_code_energie, how='left').merge(nature_code_fonction, how='left')
    data = data.merge(nature_code_installation, how='left').merge(nature_code_specification, how='left')
    print(data.shape[0], data.INCIDENT_NUMBER.nunique(), data.INSTANCE_ID.nunique())
    return data

In [15]:
drop_columns = ['CRE_DATE_date', 'UPD_DATE_date']

In [16]:
train_data = prepare_data(intervention_train_df).drop(drop_columns, axis=1)

1048030 1048030 462663


In [17]:
test_data = prepare_data(intervention_test_df).drop(drop_columns, axis=1)

226770 226770 226770
